In [1]:
pip install tkinter

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [1]:
import numpy as np
from mpi4py import MPI
from numba import cuda
import tkinter as tk
import time

In [2]:
@cuda.jit
def game_of_life_kernel(current_grid, next_grid, rows, cols):
    x, y = cuda.grid(2)
    if x >= rows or y >= cols:
        return

    live_neighbors = 0
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            neighbor_x = (x + i + rows) % rows
            neighbor_y = (y + j + cols) % cols
            live_neighbors += current_grid[neighbor_x, neighbor_y]

    if current_grid[x, y] == 1:
        if live_neighbors < 2 or live_neighbors > 3:
            next_grid[x, y] = 0
        else:
            next_grid[x, y] = 1
    else:
        if live_neighbors == 3:
            next_grid[x, y] = 1


In [3]:
def mpi_game_of_life(local_grid, num_generations, proc_grid_size):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    rows, cols = local_grid.shape

    next_grid = np.zeros_like(local_grid)

    threads_per_block = (16, 16)
    blocks_per_grid_x = (rows + threads_per_block[0] - 1) // threads_per_block[0]
    blocks_per_grid_y = (cols + threads_per_block[1] - 1) // threads_per_block[1]
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    for _ in range(num_generations):
        d_local_grid = cuda.to_device(local_grid)
        d_next_grid = cuda.to_device(next_grid)

        game_of_life_kernel[blocks_per_grid, threads_per_block](d_local_grid, d_next_grid, rows, cols)

        d_next_grid.copy_to_host(next_grid)

        if rank > 0:
            comm.Sendrecv(local_grid[0, :], dest=rank - 1, recvbuf=local_grid[-1, :])
        if rank < size - 1:
            comm.Sendrecv(local_grid[-1, :], dest=rank + 1, recvbuf=local_grid[0, :])

        local_grid = next_grid.copy()

    return local_grid


In [4]:
class GameOfLifeGUI:
    def __init__(self, root, grid_size=(128, 128), cell_size=5):
        self.root = root
        self.grid_size = grid_size
        self.cell_size = cell_size
        self.running = False

        self.canvas = tk.Canvas(self.root, width=grid_size[1]*cell_size, height=grid_size[0]*cell_size)
        self.canvas.pack()

        self.start_button = tk.Button(self.root, text="Start", command=self.start_game)
        self.start_button.pack(side=tk.LEFT)
        self.stop_button = tk.Button(self.root, text="Stop", command=self.stop_game)
        self.stop_button.pack(side=tk.LEFT)

        self.local_grid = np.random.randint(2, size=grid_size)  # Initialize with random grid
        self.update_gui()

    def update_gui(self):
        self.canvas.delete("all")
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                if self.local_grid[i, j] == 1:
                    x1 = j * self.cell_size
                    y1 = i * self.cell_size
                    x2 = x1 + self.cell_size
                    y2 = y1 + self.cell_size
                    self.canvas.create_rectangle(x1, y1, x2, y2, fill="black")

    def start_game(self):
        self.running = True
        self.run_game()

    def stop_game(self):
        self.running = False

    def run_game(self):
        if self.running:
            proc_grid_size = MPI.COMM_WORLD.Get_size()
            self.local_grid = mpi_game_of_life(self.local_grid, 1, proc_grid_size)
            self.update_gui()
            self.root.after(100, self.run_game)  # Update every 100 ms

def main():
    root = tk.Tk()
    root.title("Conway's Game of Life - MPI + GPU")

    grid_size = (128, 128)  # Example grid size
    game_gui = GameOfLifeGUI(root, grid_size)

    root.mainloop()

if __name__ == "__main__":
    main()


c:\Users\shaky\anaconda3\envs\gameofvenv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
